# Load Libraries

In [3]:
######################################################Initial Packages########################################################
#Basic Operating System Stuff
import sklearn
import os
import gc #garbage collector
import random #random seed generator
import sys
import csv
import subprocess

#Basic dataframe, array, and math stuff
import pandas as pd #data frame
import math #math functions
import numpy as np    #numerical package

#Sci-kit Learn
from sklearn.preprocessing import MinMaxScaler as mms
from sklearn.metrics import classification_report as cr
from sklearn.model_selection import train_test_split as tts

#Tensorflow
import tensorflow as tf #backend for keras
from tensorflow.python.client import device_lib #to  see if my GPU is alive!
import tensorflow.keras #keras
from tensorflow.keras import Sequential,Input,Model #pull in the sequential, input layers and a model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Embedding, Bidirectional, Dense, LSTM, Concatenate, concatenate #pull in the dense, dropout, and flatten layers
from tensorflow.keras.layers import Add, Activation, ZeroPadding2D, MaxPool2D, BatchNormalization
from tensorflow.keras.applications import EfficientNetB6#MobileNetV2
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint # ReduceLROnPlateau, use for early stopping and reduction on level-out
from tensorflow.keras import utils #Need utilities for the layers
from tensorflow.keras.utils import get_file, plot_model #To load certain files & plot models
import tensorflow.keras.backend as K #To write our own metrics and loss functions
from tensorflow.keras.preprocessing.text import Tokenizer #RNN
from tensorflow.keras.preprocessing.sequence import pad_sequences #RNN

#Graphing
import matplotlib #image save
import matplotlib.pyplot as plt #plotting
from matplotlib.pyplot import imshow #Show images
from PIL import Image, ImageFont, ImageDraw
import glob
import cv2

#Text Analysis
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import re
import spacy
from spacymoji import Emoji
import pickle
import re
import string
from string import punctuation
from num2words import num2words



#Directory

os.chdir("c:/users/lfult/documents/churnplay/")

%matplotlib inline

#Verify GPU recognition
print(device_lib.list_local_devices()) #Let's see if Python recognizes my GPU, shall we?
##############################################################################################################################

ModuleNotFoundError: No module named 'cv2'

# Memory Management

In [ ]:
#################################################Memory Mgt / Directory Load###################################################
def reset_keras():
    tf.keras.backend.clear_session #This clears the GPU session
    gc.collect()
reset_keras()            
################################################################################################################################

# Read Data 

In [ ]:
traindf=pd.read_csv('modtrain.csv', dtype='str')
testdf=pd.read_csv('modtest.csv', dtype='str')
traindf.head()
nlpdata=pd.DataFrame(pd.concat([traindf['Concatenate'], testdf['Concatenate']]))

In [ ]:
traindf.shape

In [ ]:
testdf.shape

# Text Preprocessing

In [ ]:
import nltk
import string
from num2words import num2words
from string import punctuation
from emot.emo_unicode import UNICODE_EMOJI 
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
#nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words('english'))
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def clean_html(text):
  CLEANR = re.compile('<.*?>') 
  text = re.sub(CLEANR, '', text)
  return text

def remove_stopwords(text):
    return [word for word in word_tokenize(text) if not word in stopwords]

def strip_punctuation(text):
    return ''.join(c for c in text if c not in punctuation)+' '    

def to_number(text):    
    return(re.sub(r"(\d+)", lambda x: num2words(int(x.group(0))), text))

def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

# NLP Apply PreProcessing

In [ ]:
nlpdata['Concatenate']=nlpdata['Concatenate'].str.lower()
nlpdata['Concatenate']=nlpdata['Concatenate'].astype(str).apply(clean_html)
nlpdata['Concatenate']=nlpdata['Concatenate'].astype(str).apply(remove_stopwords)
nlpdata['Concatenate']=nlpdata['Concatenate'].astype(str).apply(strip_punctuation)
nlpdata['Concatenate']=nlpdata['Concatenate'].astype(str).apply(to_number)
nlpdata['Concatenate']=nlpdata['Concatenate'].astype(str).apply(lemmatize_words)

# Split back to Training / Test Sets

In [ ]:
train=nlpdata.iloc[0:175036,]
test=nlpdata.iloc[175036:285059,]

# Write CSV

In [ ]:
train.to_csv('trainNLP.csv', index=False)
test.to_csv('testNLP.csv',index=False)

# Read CSV

In [ ]:
train=pd.read_csv("trainNLP.csv")
test=pd.read_csv("testNLP.csv")
train['label']=traindf['Exited']

# Initialize Model Settings

In [ ]:
reset_keras()
mybatch=64   #set batch size
embedding_size=512 #set out size for embedding
num_epochs=4 #set the number of epochs..
num_words = 50000     #number of words to tokenize, 50000 was start
tf.random.set_seed(1234) #set random number seed for tensorflow, python, etc.

# Make Data Sets

In [ ]:
train2=pd.read_csv('modtrain.csv')
test2=pd.read_csv('modtest.csv')

x_train1=train['Concatenate'] 
x_test1=test['Concatenate']      

x_train2=train2.iloc[:,3:12]
x_test2=test2.iloc[:, 3:12]       

y_train=train['label']

In [ ]:
x_train2=pd.get_dummies(x_train2)
x_test2=pd.get_dummies(x_test2)

# Min Max Scale for Continuous Variables

In [ ]:
transform=mms() #declare min max scaler
x_train2=transform.fit_transform(x_train2)   #fit transform training data
x_test2=transform.transform(x_test2)                                                         

# Tokenize Text Data

In [ ]:
oov_token = '<UNK>'   #out of vocabulary replacement
pad_type = 'post'     #padding type
trunc_type = 'post'   #truncation type

tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)

tokenizer.fit_on_texts(x_train1.astype('str'))
#tokenizer.fit_on_texts(x_train2.astype('str'))

# Encode training data sentences into sequences
train_sequences1 = tokenizer.texts_to_sequences(x_train1.astype('str')) 

# Get max training sequence length
maxlen1 = max([len(x) for x in train_sequences1])

# Pad the training sequences to the maximum sentence length
x_train1 = pad_sequences(train_sequences1, padding=pad_type, truncating=trunc_type, maxlen=maxlen1)

# Tokenize Test Data

In [ ]:
# Encode test data sentences into sequences
test_sequences1 = tokenizer.texts_to_sequences(x_test1.astype('str'))

# Pad the training sequences to the maximum sentence length of training set
x_test1 = pad_sequences(test_sequences1, padding=pad_type, truncating=trunc_type, maxlen=maxlen1)

# Multi-Input Model

## Input Shapes

In [ ]:
input_1 = Input(shape=(x_train1.shape[1]))
input_2 = Input(shape=(x_train2.shape[1]))

## RNN

In [ ]:
v=Embedding(len(tokenizer.word_index) + 1, embedding_size)(input_1)
v=Bidirectional(LSTM(embedding_size, return_sequences=True, dropout=.5))(v)
v=Bidirectional(LSTM(embedding_size, return_sequences=False, dropout=.2))(v)

## NN

In [ ]:
y=Dense(128)(input_2)
y=Dropout(.5)(y)
y=Dense(64)(y)
y=Dropout(.4)(y)
y=Dense(32)(y)
y=Dropout(.2)(y)

## Concatenate Models

In [ ]:
concat=tf.keras.layers.concatenate([v,y], axis=-1)
output=Dense(16,activation='relu')(concat)
output=Dropout(.1)(output)
output=Dense(8, activation='relu')(output)
output=Dense(1, activation='sigmoid')(output)
model=Model(inputs=[input_1, input_2], outputs=output)

## Compile

In [ ]:
with tf.device('/GPU:0'):    
    model = model
    SGD= tensorflow.keras.optimizers.SGD(lr=0.0001, nesterov=True)
    adam=tensorflow.keras.optimizers.Adam(lr=0.0001)
    model.compile(loss=tensorflow.keras.losses.binary_crossentropy, 
              optimizer=adam, metrics=['binary_accuracy'])
    model.summary()

## Plot Model

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True, expand_nested=False)

# Prepare for Model Run

In [ ]:
tf.config.run_functions_eagerly(True)
reset_keras()
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='binary_accuracy', verbose=2,
    save_best_only=True, mode='auto')

# Model Run

In [ ]:

#model.load_weights('best_model.hdf5')

from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
y_train=lb.fit_transform(y_train)
reset_keras()

with tf.device('/GPU:0'):    
    history=model.fit(
                  x=[x_train1, 
                     x_train2],y=y_train,
                  batch_size=mybatch,verbose=1,epochs=num_epochs, callbacks=[checkpoint], validation_split=.3) 
    
#validation_split=.1 removed after tuning models
#2 epochs works well (hyperparameter tuning)

# Predict on Test Set

In [ ]:
predict=model.predict([x_test1, x_test2])
predict1=np.squeeze(predict)
np.unique(np.round(predict1,0), return_counts=True)